# Lab 8 Neural Language Model
A language model predicts the next word in the sequence based on the specific words that have come before it in the sequence.

It is also possible to develop language models at the character level using neural networks. The benefit of character-based language models is their small vocabulary and flexibility in handling any words, punctuation, and other document structure. This comes at the cost of requiring larger models that are slower to train.

Nevertheless, in the field of neural language models, character-based models offer a lot of promise for a general, flexible and powerful approach to language modeling.

As a prerequisite for the lab, make sure to pip install:
- keras
- tensorflow
- h5py

# Source Text Creation

To start out with, we'll be using a simple nursery rhyme. It's quite short so we can actually train something on your CPU and see relatively interesting results. Please copy and paste the following text in a text file and save it as "rhyme.txt". Place this in the same directory as this jupyter notebook:

In [ ]:
!pip install tensorflow
!pip install keras
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
s='Mary had a little lamb, little lamb, little lamb\
Mary had a little lamb, its fleece was white as snow\
And everywhere that Mary went. Mary went. Mary went\
And everywhere that Mary went, the lamb was sure to go\
It followed her to school one day, school one day, school one day\
It followed her to school one day, which was against the rule\
It made the children laugh and play, laugh and play, laugh and play\
It made the children laugh and play to see the lamb at school\
And so the teacher sent it out, sent it out, sent it out\
And so the teacher sent it out, but still it lingered near\
It stood and waited round about, round about, round about\
It stood and waited round about, till Mary did appear\''

with open('rhymes.txt','w') as f:
  f.write(s)

    Sing a song of sixpence,
    A pocket full of rye.
    Four and twenty blackbirds,
    Baked in a pie.

    When the pie was opened
    The birds began to sing;
    Wasn’t that a dainty dish,
    To set before the king.

    The king was in his counting house,
    Counting out his money;
    The queen was in the parlour,
    Eating bread and honey.

    The maid was in the garden,
    Hanging out the clothes,
    When down came a blackbird
    And pecked off her nose.

# Sequence Generation

A language model must be trained on the text, and in the case of a character-based language model, the input and output sequences must be characters.

The number of characters used as input will also define the number of characters that will need to be provided to the model in order to elicit the first predicted character.

After the first character has been generated, it can be appended to the input sequence and used as input for the model to generate the next character.

Longer sequences offer more context for the model to learn what character to output next but take longer to train and impose more burden on seeding the model when generating text.

We will use an arbitrary length of 10 characters for this model.

There is not a lot of text, and 10 characters is a few words.

We can now transform the raw text into a form that our model can learn; specifically, input and output sequences of characters.

In [ ]:
#load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [ ]:
#load text
raw_text = load_doc('rhymes.txt')
print(raw_text)

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Mary had a little lamb, little lamb, little lambMary had a little lamb, its fleece was white as snowAnd everywhere that Mary went. Mary went. Mary wentAnd everywhere that Mary went, the lamb was sure to goIt followed her to school one day, school one day, school one dayIt followed her to school one day, which was against the ruleIt made the children laugh and play, laugh and play, laugh and playIt made the children laugh and play to see the lamb at schoolAnd so the teacher sent it out, sent it out, sent it outAnd so the teacher sent it out, but still it lingered nearIt stood and waited round about, round about, round aboutIt stood and waited round about, till Mary did appear'
Total Sequences: 674


In [ ]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

# Train a Model
In this section, we will develop a neural language model for the prepared sequence data.

The model will read encoded characters and predict the next character in the sequence. A Long Short-Term Memory recurrent neural network hidden layer will be used to learn the context from the input sequence in order to make the predictions.

In [ ]:
from numpy import array
from pickle import dump
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
# load

in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

The sequences of characters must be encoded as integers.This means that each unique character will be assigned a specific integer value and each sequence of characters will be encoded as a sequence of integers. We can create the mapping given a sorted set of unique characters in the raw input data. The mapping is a dictionary of character values to integer values.

Next, we can process each sequence of characters one at a time and use the dictionary mapping to look up the integer value for each character. The result is a list of integer lists.

We need to know the size of the vocabulary later. We can retrieve this as the size of the dictionary mapping.

In [ ]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
old_sequences = sequences.copy()
temp = old_sequences.copy()
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 29


The model is defined with an input layer that takes sequences that have 10 time steps and 38 features for the one hot encoded input sequences. Rather than specify these numbers, we use the second and third dimensions on the X input data. This is so that if we change the length of the sequences or size of the vocabulary, we do not need to change the model definition.

The model has a single LSTM hidden layer with 75 memory cells. The model has a fully connected output layer that outputs one vector with a probability distribution across all characters in the vocabulary. A softmax activation function is used on the output layer to ensure the output has the properties of a probability distribution.

The model is learning a multi-class classification problem, therefore we use the categorical log loss intended for this type of problem. The efficient Adam implementation of gradient descent is used to optimize the model and accuracy is reported at the end of each batch update. The model is fit for 50 training epochs.

# To Do:
- Try different numbers of memory cells
- Try different types and amounts of recurrent and fully connected layers
- Try different lengths of training epochs
- Try different sequence lengths and pre-processing of data
- Try regularization techniques such as Dropout

In [ ]:

# define model
model = Sequential()
model.add(LSTM(300, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(200, activation='tanh'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='tanh'))

model.add(Dense(vocab_size, activation='softmax'))


print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model.fit(X, y, epochs=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 300)               396000    
                                                                 
 dense (Dense)               (None, 200)               60200     
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 29)                1479      
                                                                 
Total params: 482,829
Trainable params: 482,829
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
22/22 [==========================

In [ ]:
# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

# Generating Text

We must provide sequences of 10 characters as input to the model in order to start the generation process. We will pick these manually. A given input sequence will need to be prepared in the same way as preparing the training data for the model. 

In [ ]:
from pickle import load
import numpy as np
from keras.models import load_model
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = np.argmax(model.predict(encoded), axis=-1)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

Running the example generates three sequences of text.

The first is a test to see how the model does at starting from the beginning of the rhyme. The second is a test to see how well it does at beginning in the middle of a line. The final example is a test to see how well it does with a sequence of characters never seen before.

In [ ]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Mary had a', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'It followe', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

1/1 [==============================] - 0s 23ms/step
Mary had a little lamb, little
1/1 [==============================] - 0s 21ms/step
It followed her to school one 
1/1 [==============================] - 0s 22ms/step
hello worl th lut and ited reu


If the results aren't satisfactory, try out the suggestions above or these below:
- Padding. Update the example to provides sequences line by line only and use padding to fill out each sequence to the maximum line length.
- Sequence Length. Experiment with different sequence lengths and see how they impact the behavior of the model.
- Tune Model. Experiment with different model configurations, such as the number of memory cells and epochs, and try to develop a better model for fewer resources.


# Deliverables to receive credit

1. (4 points) Optimize the cells above to tune the model so that it generates text that closely resembles the orginal line from the rhyme, or at least generates sensible words. It's okay if the third example using unseen text still looks somewhat strange  though. Again, this is a toy problem, as language models require a lot of computation. This toy problem is great for rapid experimentation to explore different aspects of deep learning language models.
2. (3 points) Write a function to split the text corpus file into training and validation and pipe the validation data into the model.fit() function to be able to track validation error per epoch. Lookup Keras documentation to see how this is handled.
3. (3 points) Write a summary (methods and results) in the cells below of the different things you applied. You must include your intuitions behind what did work and what did not work well.
4. (Extra Credit 2.5 points) Do something even more interesting. Try a different source text. Train a word-level model. We'll leave it up to your creativity to explore and write a summary of your methods and results.


ANSWER TO Q3.

3)
I attemped to use more layers in order to try to increase the accuracy of the model. This did help with the accuracy because the RNN had more information and weights to use when predicting characters.
I used different activation methods which greatly increased the speed and accuracy of the model after adding the second layer because relu was more applicable.
I also increased the number of epochs and the input node size so that there would be more connections in the neural network and so the training would run more times. This would strengthen the proper weights in the NN so that we would get a better model.
I increased the sequence length and got a worse model, so I reverted back to 10. This might have happened because when I increased the length, the computer power was more, so I had to decrease the number of epochs so it would run in a reasonable time, which would reduce the accuracy. Similarly, I attempted to use Dropout regularization, but it did not end up increasing the accuracy or predicting a better set of characters.
I looked at creating another model and merging the two, however I was not able to get it working properly.

In [ ]:
def split_text(text):
  np.random.seed(42)

  split = 0
  while  split > 0.9 or  split < 0.4:
    split = np.random.rand()
  return text[:int(split*len(text))], text[int(split*len(text)):]
  


In [ ]:
#print(old_sequences)
new_train, new_test = split_text(old_sequences)
X_train, y_train = new_train[:,:-1], new_train[:,-1]
X_test, y_test = new_test[:,:-1], new_test[:,-1]


In [ ]:
new_train = [to_categorical(x, num_classes=vocab_size) for x in X_train]
new_test = [to_categorical(x, num_classes=vocab_size) for x in X_test]
#old_sequences
X_train = array(new_train)
y_train = to_categorical(y_train, num_classes=vocab_size)
X_test = array(new_test)
y_test = to_categorical(y_test, num_classes=vocab_size)

In [ ]:
# define model
model2 = Sequential()
model2.add(LSTM(300, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
#model2.add(LSTM(300, return_sequences=True))
model2.add(LSTM(300))
model2.add(Dense(200, activation='relu'))
model2.add(Dropout(0.4))
#model2.add(Dense(100, activation='relu'))
#model2.add(Dense(50, activation='relu'))
model2.add(Dense(vocab_size, activation='softmax'))


print(model2.summary())
# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history2=model2.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test))

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_50 (LSTM)              (None, 10, 300)           396000    
                                                                 
 lstm_51 (LSTM)              (None, 300)               721200    
                                                                 
 dense_77 (Dense)            (None, 200)               60200     
                                                                 
 dropout_25 (Dropout)        (None, 200)               0         
                                                                 
 dense_78 (Dense)            (None, 29)                5829      
                                                                 
Total params: 1,183,229
Trainable params: 1,183,229
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
16/16 [===================

In [ ]:
# save the model to file
model2.save('model2.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

In [ ]:
# load the model
model2 = load_model('model2.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [ ]:
# test start of rhyme


print(generate_seq(model2, mapping, 10, 'Mary had a', 20))
# test mid-line
print(generate_seq(model2, mapping, 10, 'It followe', 20))
# test not in original
print(generate_seq(model2, mapping, 10, 'hello worl', 20))

1/1 [==============================] - 0s 18ms/step
Mary had a little lamb, little
1/1 [==============================] - 0s 22ms/step
It followed her to school one 
1/1 [==============================] - 0s 21ms/step
hello worleh  ttaigy eeswnauhe
